In [ ]:
!pip install sentence-transformers
!pip install faiss-cpu
!pip install PyPDF2
!pip install torch torchvision accelerate transformers
!pip install huggingface-hub
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.2 MB/s eta 0:00:00


In [ ]:
!pip install discord.py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.0 MB/s eta 0:00:00


In [ ]:
!pip install -U google-generativeai

In [ ]:
import google.generativeai as genai
GEMINI_API_KEY = "Gemini_key_api"
genai.configure(api_key=GEMINI_API_KEY)

# List models
for m in genai.list_models():
    print(m.name, m.supported_generation_methods)


models/embedding-gecko-001 ['embedText', 'countTextTokens']
models/gemini-2.5-pro-preview-03-25 ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-flash-preview-05-20 ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-flash ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-flash-lite-preview-06-17 ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-pro-preview-05-06 ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-pro-preview-06-05 ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-pro ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-exp ['generateContent', 'countTokens', 'bidiGenerateContent']
models/gemini-2.0-flash ['generateContent', '

In [7]:
# 🪴 Step 1: Install dependencies
!pip install -q discord.py google-generativeai faiss-cpu PyPDF2 sentence-transformers

# 🪴 Step 2: Imports
import os
import discord
from discord.ext import commands
import google.generativeai as genai
from PyPDF2 import PdfReader
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from google.colab import files

# 🪴 Step 3: API Keys
DISCORD_BOT_TOKEN = "YOUR_DISCORD_BOT_TOKEN"
GEMINI_API_KEY = "YOUR_GEMINI_KEY"

os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY
genai.configure(api_key=GEMINI_API_KEY)

# 🪴 Step 4: Initialize models
EMBED_MODEL = "models/text-embedding-004"
LLM_MODEL = "models/gemini-2.5-pro"
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# 🪴 Step 5: Load & Embed PDF
def load_and_embed_pdf(pdf_path):
    text = ""
    reader = PdfReader(pdf_path)
    for page in reader.pages:
        text += page.extract_text() or ""

    # Split text into chunks
    chunks = [text[i:i+500] for i in range(0, len(text), 500)]

    # Generate embeddings
    embeddings = embedder.encode(chunks)

    # Create FAISS index
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(np.array(embeddings))

    return chunks, index

# 🪴 Step 6: RAG Search
def retrieve(query, chunks, index, top_k=3):
    query_embedding = embedder.encode([query])
    D, I = index.search(query_embedding, top_k)
    return [chunks[i] for i in I[0]]

# 🪴 Step 7: Ask Gemini
def ask_gemini(prompt):
    model = genai.GenerativeModel(LLM_MODEL)
    response = model.generate_content(prompt)
    return response.text.strip()

# 🪴 Step 8: Discord Bot Setup
intents = discord.Intents.default()
intents.message_content = True
bot = commands.Bot(command_prefix="!", intents=intents)

@bot.event
async def on_ready():
    print(f"✅ Logged in as {bot.user}")

# 🪴 Step 9: Global variables for PDF data
pdf_chunks = []
pdf_index = None
pdf_loaded = False

@bot.command()
async def loadpdf(ctx):
    """Load the uploaded PDF for RAG"""
    global pdf_chunks, pdf_index, pdf_loaded

    try:
        # Upload PDF file through Discord
        if not ctx.message.attachments:
            await ctx.send("⚠️ Please upload a PDF file with this command!")
            return

        attachment = ctx.message.attachments[0]
        if not attachment.filename.lower().endswith('.pdf'):
            await ctx.send("⚠️ Please upload a PDF file!")
            return

        # Download the file
        file_path = f"/content/{attachment.filename}"
        await attachment.save(file_path)

        # Load and embed PDF
        pdf_chunks, pdf_index = load_and_embed_pdf(file_path)
        pdf_loaded = True

        await ctx.send(f"📄 PDF '{attachment.filename}' loaded and embedded successfully!")

    except Exception as e:
        await ctx.send(f"❌ Error loading PDF: {str(e)}")

@bot.command()
async def ask(ctx, *, query: str):
    """Ask a question using RAG"""
    global pdf_loaded

    if not pdf_loaded:
        await ctx.send("⚠️ Please load a PDF first using !loadpdf (with a PDF file attached)")
        return

    try:
        # Retrieve relevant chunks
        retrieved = retrieve(query, pdf_chunks, pdf_index)
        context = "\n".join(retrieved)

        # Create prompt for Gemini
        prompt = f"""Use the context below to answer the question. If the answer cannot be found in the context, say so.

Context:
{context}

Question: {query}

Answer:"""

        # Get answer from Gemini
        answer = ask_gemini(prompt)

        # Send response (split if too long for Discord)
        if len(answer) > 2000:
            chunks = [answer[i:i+2000] for i in range(0, len(answer), 2000)]
            for i, chunk in enumerate(chunks):
                await ctx.send(f"💬 **Answer (Part {i+1}):** {chunk}")
        else:
            await ctx.send(f"💬 **Answer:** {answer}")

    except Exception as e:
        await ctx.send(f"❌ Error processing your question: {str(e)}")

@bot.command()
async def status(ctx):
    """Check if PDF is loaded"""
    if pdf_loaded:
        await ctx.send("✅ PDF is loaded and ready for questions!")
    else:
        await ctx.send("❌ No PDF loaded. Use !loadpdf with a PDF file attachment.")

@bot.command()
async def ping(ctx):
    """Check bot latency"""
    latency = round(bot.latency * 1000)
    await ctx.send(f"🏓 Pong! Latency: {latency}ms")

# 🪴 Step 10: Manual PDF upload for Colab
def upload_and_load_pdf():
    """Manual PDF upload function for Colab"""
    global pdf_chunks, pdf_index, pdf_loaded

    print("📤 Please upload your PDF file:")
    uploaded = files.upload()

    if not uploaded:
        print("❌ No file uploaded!")
        return

    filename = list(uploaded.keys())[0]
    full_path = f"/content/{filename}"

    try:
        pdf_chunks, pdf_index = load_and_embed_pdf(full_path)
        pdf_loaded = True
        print(f"✅ PDF '{filename}' loaded successfully!")
        print(f"📊 Number of chunks: {len(pdf_chunks)}")
    except Exception as e:
        print(f"❌ Error loading PDF: {e}")

# 🪴 Step 11: Run the bot
print("🤖 Starting Discord Bot...")

# Option to upload PDF before starting bot
print("\nWould you like to upload a PDF now?")
upload_choice = input("Type 'yes' to upload or 'no' to skip: ").lower().strip()

if upload_choice in ['yes', 'y']:
    upload_and_load_pdf()

try:
    bot.run(DISCORD_BOT_TOKEN)
except Exception as e:
    print(f"❌ Error running bot: {e}")

🤖 Starting Discord Bot...

Would you like to upload a PDF now?
Type 'yes' to upload or 'no' to skip: yes
📤 Please upload your PDF file:


2025-10-05 16:47:25 INFO     discord.client logging in using static token
2025-10-05 16:47:25 INFO     discord.client logging in using static token
2025-10-05 16:47:25 INFO     discord.client logging in using static token
2025-10-05 16:47:25 INFO     discord.client logging in using static token
2025-10-05 16:47:25 INFO     discord.client logging in using static token
INFO:discord.client:logging in using static token


Saving UHV.pdf to UHV (3).pdf
✅ PDF 'UHV (3).pdf' loaded successfully!
📊 Number of chunks: 5


2025-10-05 16:47:26 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: 67243721590b5a568ff6f0216a39a1c1).
2025-10-05 16:47:26 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: 67243721590b5a568ff6f0216a39a1c1).
2025-10-05 16:47:26 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: 67243721590b5a568ff6f0216a39a1c1).
2025-10-05 16:47:26 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: 67243721590b5a568ff6f0216a39a1c1).
2025-10-05 16:47:26 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: 67243721590b5a568ff6f0216a39a1c1).
INFO:discord.gateway:Shard ID None has connected to Gateway (Session ID: 67243721590b5a568ff6f0216a39a1c1).


✅ Logged in as SunnyBot#5896
